In [1]:
# Neo4j connection configuration
# This cell reads values from a local .env (if present) but allows students to override them interactively.
import os
from dotenv import load_dotenv
import getpass

load_dotenv()  # optional: reads neo4j connection info from .env into environment variables
# Default values read from environment (if set)
default_uri = os.getenv('NEO4J_URI', '')
default_user = os.getenv('NEO4J_USERNAME', '')
default_pass = os.getenv('NEO4J_PASSWORD', '')

print('If you have values in .env they will be used as defaults. You may press Enter to accept the default.')
uri = default_uri or input(f'Neo4j URI [{default_uri}]: ')
user = default_user or input(f'Username [{default_user}]: ')
# use getpass for interactive password entry (safer than plain input)
pw_prompt = 'Password (leave empty to use value from .env): '
password = default_pass or getpass.getpass(pw_prompt)

# Save into variables used by the rest of the notebook
NEO4J_URI = uri
NEO4J_USER = user
NEO4J_PASS = password

print('Configuration set. Do not share passwords in public notebooks.')

If you have values in .env they will be used as defaults. You may press Enter to accept the default.
Configuration set. Do not share passwords in public notebooks.


# Neo4j basics — workshop

Welcome! This short workshop walks through basic Neo4j concepts and gives hands-on practice with nodes, relationships, queries, and simple modeling.

Goals:
- Connect to a Neo4j database from Python.
- Create simple nodes and relationships.
- Run basic Cypher queries (MATCH, RETURN, aggregation).
- Create an index/constraint and clean up sample data.

Important: the connection values you entered in the previous cell will be used. If your environment also has a `.env` file, the values were used as defaults.

In [2]:
# Test connection and show server info
from neo4j import GraphDatabase
from neo4j_viz import GraphViz

def test_connection(uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    try:
        with driver.session() as session:
            try:
                comp = list(session.run(
                    "CALL dbms.components() YIELD name, versions, edition RETURN name, versions, edition"
                ))
                print('dbms.components():')
                for r in comp:
                    print('-', r.data())
            except Exception as e:
                print('Could not call dbms.components():', e)
            # Try SHOW DATABASES in case the server supports multiple DBs
            try:
                dbs = list(session.run("SHOW DATABASES"))
                print('SHOW DATABASES: found', len(dbs))
            except Exception as e:
                print('SHOW DATABASES not available or no privileges:', e)
    finally:
        driver.close()

# Run test (uses variables set in the first cell)
print('Testing connection to', NEO4J_URI)
test_connection(NEO4J_URI, NEO4J_USER, NEO4J_PASS)

ImportError: cannot import name 'GraphViz' from 'neo4j_viz' (/Users/zziang/Documents/projects/abds_nosql_course/.venv/lib/python3.13/site-packages/neo4j_viz/__init__.py)

In [5]:
# Basic Neo4j statistics: total nodes, total relationships, top labels and relationship types
from neo4j import GraphDatabase
from neo4j_viz.neo4j import from_neo4j

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        record = result.single()
        value = record.value() if record else None
        summary = result.consume()
        print(f"Query executed in {summary.result_available_after} ms, ")
        return value


# get total nodes and relationships
result = run_query("MATCH (n) RETURN count(n) AS cnt")
print("Total nodes:", result)

result = run_query("MATCH ()-[r]->() RETURN count(r) AS cnt")
print("Total relationships:", result)


Query executed in 1 ms, 
Total nodes: 47034
Query executed in 17 ms, 
Total relationships: 409059


### Exercise 1 (first task) — Build your node

We design a graph model to describe this lecture

```mermaid
---
title: The course
---
erDiagram
    MeetingRoom {
      id MP163
      building ARC
      floor plaza
    }
    Lecture {
      program ABDS
      series DataWrangling
      index n26
    }
    MeetingRoom ||--o{ Lecture: HOSTS
    Lecturer {
        uid zziang
        dept InformationService
    }
    Lecturer ||--o{ Lecture: GIVES
```

In [ ]:
lecture_query = ('MATCH (n)-[r:Lecture]->(m) '
                 'RETURN n, r, m LIMIT 200')
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))
viz = GraphViz(driver)
drv = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))
nodes = {}
rels = []
with drv.session() as session:
    try:
        res = session.run(q)
    except Exception as e:
        print('Query failed:', e)
        raise
    for rec in res:
        d = rec.data()
        # add source node
        nid = d.get('n_id')
        if nid is not None and nid not in nodes:
            nodes[nid] = {'id': nid, 'labels': d.get('n_labels') or [], 'props': d.get('n_props') or {}}
        # add target node
        mid = d.get('m_id')
        if mid is not None and mid not in nodes:
            nodes[mid] = {'id': mid, 'labels': d.get('m_labels') or [], 'props': d.get('m_props') or {}}
        # add relationship
        rid = d.get('r_id')
        if rid is not None:
            rels.append({'id': rid, 'src': nid, 'tgt': mid, 'type': d.get('r_type'), 'props': d.get('r_props') or {}})

nodes_list = list(nodes.values())
print(f'Prepared {len(nodes_list)} nodes and {len(rels)} relationships for visualization')

# Try neo4j_viz first
try:
    nv = importlib.import_module('neo4j_viz')
    if hasattr(nv, 'draw'):
        nv.draw(nodes_list, rels)
    elif hasattr(nv, 'viz'):
        nv.viz(nodes_list, rels)
    else:
        raise ImportError('neo4j_viz missing known entrypoint')
except Exception as e:
    print('neo4j_viz not available or failed:', e)
    print('Falling back to networkx visualization')
    try:
        import networkx as nx
        import matplotlib.pyplot as plt
        G = nx.DiGraph()
        for n in nodes_list:
            label = n['props'].get('name') or (n['labels'][0] if n['labels'] else str(n['id']))
            G.add_node(n['id'], label=label)
        for r in rels:
            G.add_edge(r['src'], r['tgt'], label=r.get('type'))
        pos = nx.spring_layout(G)
        plt.figure(figsize=(8,8))
        nx.draw(G, pos, with_labels=True, labels=nx.get_node_attributes(G, 'label'), node_size=700, node_color='lightgreen')
        edge_labels = nx.get_edge_attributes(G, 'label')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        plt.show()
    except Exception as ex:
        print('Networkx fallback failed:', ex)

drv.close()